In [1]:
import sys
import torch
from transformers import BertTokenizer, BertTokenizerFast, BertModel, BertConfig
from tokenizers import BertWordPieceTokenizer


sys.path.insert(0, '../')
from model import load_model
from config import Config, TrainArgs, ModelType

In [2]:
# for train
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments



In [3]:
config = BertConfig.from_pretrained(ModelType.BertMultiLingual)

tokenizer = BertTokenizer.from_pretrained(ModelType.BertMultiLingual)
model = BertModel.from_pretrained(ModelType.BertMultiLingual, config=config)


In [ ]:
training_args = TrainingArguments(**TrainArgs.Base)

data_collaor = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)